In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from skimage.io import imread
import os
import sys


# Current folder as project_root
# Going three layers of folders upwards to src
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
src_dir      = os.path.join(project_root)

# Add the src directory to the Python module search path
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

# Import functions from src
import importlib
import src.Dice_Score
importlib.reload(src.Dice_Score)

from src.pre_processing import gammacorrection
from src.Complete_Otsu_Global import otsu_threshold_skimage_like
from src.Dice_Score import dice_score




# Process single image and its groundtruth
def process_single(img_path: Path, gt_path: Path, g=None) -> float:
    """ 
    Reads one image and corresponding groundtruth, proccesses, segments and computes dice score for one image.
    """
    # Reads image and reads, binarizes groundtruth
    img = imread(img_path, as_gray=True)
    gt = imread(gt_path, as_gray=True)        

    # Gamma transformation
    img_gamma = gammacorrection(img, gamma=g)
    
    # 1nd Otsu
    binary1 = otsu_threshold_skimage_like(img_gamma)

    # 5. Compute Dice score
    return dice_score(binary1.flatten(), gt.flatten())

# -------------------------------------------------------------------
# Mainroutine: Going through all image-gt pairs and collect all dice scores of data set: NIH3T3
# -------------------------------------------------------------------

# Define gamma values to test (e.g., from 0.1 to 2.0 in steps of 0.1)
gamma_values = np.arange(0.1, 2.1, 0.1)

# Set paths
img_dir = Path(os.path.join(project_root, "data-git", "NIH3T3", "img"))
gt_dir = Path(os.path.join(project_root, "data-git", "NIH3T3", "gt"))

# Loop over gamma values
dice_score_means_NIH3T3 = []

for gamma in gamma_values:
    dice_scores = []

    for img_file in sorted(img_dir.glob("dna-*.png")):
        idx = img_file.stem.split('-')[-1]
        gt_file = gt_dir / f"{idx}.png"

        # Compute Dice score for current image and gamma
        score = process_single(img_file, gt_file, g=gamma)
        dice_scores.append(score)

    # Convert to numpy array and print mean
    dice_scores = np.array(dice_scores)
    dice_score_mean = np.mean(dice_scores)
    print(f"Gamma {gamma:.2f} → Mean Dice: {dice_scores.mean():.4f}")
    dice_score_means_NIH3T3.append(dice_score_mean)

np.save('Dice_scores/NIH3T3_dice_means_gamma', dice_score_means_NIH3T3)
    
# NEXT

# Set paths
# Set paths
img_dir = Path(os.path.join(project_root, "data-git", "N2DL-HeLa", "img"))
gt_dir = Path(os.path.join(project_root, "data-git", "N2DL-HeLa", "gt"))

# Loop over gamma values
dice_score_means_N2DL_HeLa = []

for gamma in gamma_values:
    dice_scores = []

    for img_file in sorted(img_dir.glob("t-*.tif")):
        idx = img_file.stem.split('-')[-1]
        gt_file = gt_dir / f"man_seg{idx}.tif"

        # Compute Dice score for current image and gamma
        score = process_single(img_file, gt_file, g=gamma)
        dice_scores.append(score)

    # Convert to numpy array and print mean
    dice_scores = np.array(dice_scores)
    dice_score_mean = np.mean(dice_scores)
    print(f"Gamma {gamma:.2f} → Mean Dice: {dice_scores.mean():.4f}")
    dice_score_means_N2DL_HeLa.append(dice_score_mean)

np.save('Dice_scores/N2DL-HeLa_dice_means_gamma', dice_score_means_N2DL_HeLa)


# NEXT

# Set paths
img_dir = Path(os.path.join(project_root, "data-git", "N2DH-GOWT1", "img"))
gt_dir = Path(os.path.join(project_root, "data-git", "N2DH-GOWT1", "gt"))

# Loop over gamma values
dice_score_means_N2DH_GOWT1 = []

for gamma in gamma_values:
    dice_scores = []

    for img_file in sorted(img_dir.glob("t-*.tif")):
        idx = img_file.stem.split('-')[-1]
        gt_file = gt_dir / f"man_seg{idx}.tif"

        # Compute Dice score for current image and gamma
        score = process_single(img_file, gt_file, g=gamma)
        dice_scores.append(score)

    # Convert to numpy array and print mean
    dice_scores = np.array(dice_scores)
    dice_score_mean = np.mean(dice_scores)
    print(f"Gamma {gamma:.2f} → Mean Dice: {dice_scores.mean():.4f}")
    dice_score_means_N2DH_GOWT1.append(dice_score_mean)

np.save('Dice_scores/N2DH-GOWT1_dice_means_gamma', dice_score_means_N2DH_GOWT1)

ValueError: Prediction and ground truth images must have the same shape.

In [ ]:
# Find best gamma for otsu local

import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from skimage.io import imread
import os
import sys


# Current folder as project_root
# Going three layers of folders upwards to src
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
src_dir      = os.path.join(project_root)

# Add the src directory to the Python module search path
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

# Import functions from src
import importlib
import src.Dice_Score
importlib.reload(src.Dice_Score)

from src.pre_processing import gammacorrection
from src.Complete_Otsu_Global import otsu_threshold_skimage_like
from src.Dice_Score import dice_score
from src.Otsu_Local import local_otsu_fast



# Process single image and its groundtruth
def process_single(img_path: Path, gt_path: Path, g=None, r=15) -> float:
    """ 
    Reads one image and corresponding groundtruth, proccesses, segments and computes dice score for one image.
    """
    # Reads image and reads, binarizes groundtruth
    img = imread(img_path, as_gray=True)
    gt = imread(gt_path, as_gray=True)        

    # Gamma transformation
    img_gamma = gammacorrection(img, gamma=g)
    
    # 1nd Otsu
    t = local_otsu_fast(img_gamma, radius=r)
    binary = (img_gamma > t).astype(np.uint8)
    
    # 5. Compute Dice score
    return dice_score(binary.flatten(), gt.flatten())

# -------------------------------------------------------------------
# Mainroutine: Going through all image-gt pairs and collect all dice scores of data set: NIH3T3
# -------------------------------------------------------------------

# Define gamma values to test (e.g., from 0.1 to 2.0 in steps of 0.1)
gamma_values = np.arange(0.1, 2.1, 0.1)

# Set paths
img_dir = Path(os.path.join(project_root, "data-git", "NIH3T3", "img"))
gt_dir = Path(os.path.join(project_root, "data-git", "NIH3T3", "gt"))

# Loop over gamma values
dice_score_means_NIH3T3 = []

for gamma in gamma_values:
    dice_scores = []

    for img_file in sorted(img_dir.glob("dna-*.png")):
        idx = img_file.stem.split('-')[-1]
        gt_file = gt_dir / f"{idx}.png"

        # Compute Dice score for current image and gamma
        score = process_single(img_file, gt_file, g=gamma)
        dice_scores.append(score)

    # Convert to numpy array and print mean
    dice_scores = np.array(dice_scores)
    dice_score_mean = np.mean(dice_scores)
    print(f"Gamma {gamma:.2f} → Mean Dice: {dice_scores.mean():.4f}")
    dice_score_means_NIH3T3.append(dice_score_mean)

np.save('Dice_scores/NIH3T3_dice_means_gamma_local', dice_score_means_NIH3T3)
    
# NEXT

# Set paths
# Set paths
img_dir = Path(os.path.join(project_root, "data-git", "N2DL-HeLa", "img"))
gt_dir = Path(os.path.join(project_root, "data-git", "N2DL-HeLa", "gt"))

# Loop over gamma values
dice_score_means_N2DL_HeLa = []

for gamma in gamma_values:
    dice_scores = []

    for img_file in sorted(img_dir.glob("t-*.tif")):
        idx = img_file.stem.split('-')[-1]
        gt_file = gt_dir / f"man_seg{idx}.tif"

        # Compute Dice score for current image and gamma
        score = process_single(img_file, gt_file, g=gamma)
        dice_scores.append(score)

    # Convert to numpy array and print mean
    dice_scores = np.array(dice_scores)
    dice_score_mean = np.mean(dice_scores)
    print(f"Gamma {gamma:.2f} → Mean Dice: {dice_scores.mean():.4f}")
    dice_score_means_N2DL_HeLa.append(dice_score_mean)

np.save('Dice_scores/N2DL-HeLa_dice_means_gamma_local', dice_score_means_N2DL_HeLa)


# NEXT

# Set paths
img_dir = Path(os.path.join(project_root, "data-git", "N2DH-GOWT1", "img"))
gt_dir = Path(os.path.join(project_root, "data-git", "N2DH-GOWT1", "gt"))

# Loop over gamma values
dice_score_means_N2DH_GOWT1 = []

for gamma in gamma_values:
    dice_scores = []

    for img_file in sorted(img_dir.glob("t-*.tif")):
        idx = img_file.stem.split('-')[-1]
        gt_file = gt_dir / f"man_seg{idx}.tif"

        # Compute Dice score for current image and gamma
        score = process_single(img_file, gt_file, g=gamma)
        dice_scores.append(score)

    # Convert to numpy array and print mean
    dice_scores = np.array(dice_scores)
    dice_score_mean = np.mean(dice_scores)
    print(f"Gamma {gamma:.2f} → Mean Dice: {dice_scores.mean():.4f}")
    dice_score_means_N2DH_GOWT1.append(dice_score_mean)

np.save('Dice_scores/N2DH-GOWT1_dice_means_gamma_local', dice_score_means_N2DH_GOWT1)

In [ ]:
import os
import sys
import numpy as np
from pathlib import Path
from skimage.io import imread

# Current folder as project_root (3 levels up)
project_root = os.path.abspath(os.path.join(os.getcwd(), "..", "..", ".."))
src_dir      = os.path.join(project_root)

# Add the src directory to the Python module search path
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

# Import functions from src
import importlib
import src.Dice_Score
importlib.reload(src.Dice_Score)

from src.pre_processing import gammacorrection
from src.Complete_Otsu_Global import otsu_threshold_skimage_like
from src.Dice_Score import dice_score

# Provided loader functions (unchanged)
from src.imread_all import (
    load_nih3t3_images,
    load_n2dl_hela_images,
    load_n2dh_gowt1_images,
)

# New function to calculate Dice scores for all images (global threshold, fixed gamma 0.6)
def calculate_dice_scores_gamma_global(imgs, gts):
    """
    Process all images and corresponding ground truths to compute a list of Dice scores.

    Args:
        imgs (list of np.ndarray): Grayscale input images.
        gts (list of np.ndarray): Corresponding ground-truth masks.

    Returns:
        dice_scores (list of float): Dice scores for each image-groundtruth pair.
    """
    dice_scores = []

    for img, gt in zip(imgs, gts):
        # binarize groundtruth
        gt_bin = 1 - ((gt / gt.max()) == 0)

        # gamma correction
        img_gamma = gammacorrection(img, gamma=0.6)

        # global Otsu thresholding
        t = otsu_threshold_skimage_like(img_gamma)
        binary1 = (img_gamma > t)

        # calculate Dice score
        score = dice_score(binary1.flatten(), gt_bin.flatten())
        dice_scores.append(score)

    return dice_scores

# Function to evaluate multiple datasets using the new Dice calculator
def evaluate_datasets(datasets, gamma_values, output_dir='Dice_scores'):
    """
    Runs evaluation loop over given datasets and gamma sweep,
    combines all mean Dice scores into one 2D array (columns: datasets,
    rows: gamma values) and returns it.
    """
    all_means_list = []
    names = []

    for name, loader in datasets:
        # Load images & ground truths
        imgs, gts, _, _ = loader(
            base_path=os.path.join(project_root, "data-git", name)
        )
        names.append(name)

        dice_means = []
        print(f"=== Processing {name} ===")
        for gamma in gamma_values:
            # Calculate dice scores for all images at fixed gamma (0.6)
            scores = calculate_dice_scores_gamma_global(imgs, gts)
            mean_score = np.mean(scores)
            print(f"Gamma {gamma:.2f} → Mean Dice: {mean_score:.4f}")
            dice_means.append(mean_score)

        all_means_list.append(dice_means)

    # Combine vectors and transpose: shape (gammas, datasets)
    all_means = np.vstack(all_means_list).T
    return all_means

# Gamma values to test
gamma_values = np.arange(0.1, 2.1, 0.1)

# Define datasets and call evaluation function
datasets = [
    ('N2DH-GOWT1', load_n2dh_gowt1_images),
    ('N2DL-HeLa', load_n2dl_hela_images),
    ('NIH3T3', load_nih3t3_images),
]

all_means_gamma = evaluate_datasets(datasets, gamma_values)

# Function to find the optimal gamma value
def find_best_gamma(all_means, gamma_values):
    """
    Finds the gamma that maximizes the average Dice score across datasets.
    """
    mean_across = np.mean(all_means, axis=1)
    best_idx = np.argmax(mean_across)
    best_gamma = gamma_values[best_idx]
    best_dice = mean_across[best_idx]
    return best_gamma, best_dice

# Find and display best gamma
best_gamma, best_dice = find_best_gamma(all_means_gamma, gamma_values)
print(f"Optimal gamma value: {best_gamma:.2f} Mean Dice = {best_dice:.4f}")


=== Processing N2DH-GOWT1 ===
Gamma 0.10 → Mean Dice: 0.2934
Gamma 0.20 → Mean Dice: 0.2934
Gamma 0.30 → Mean Dice: 0.2934
Gamma 0.40 → Mean Dice: 0.2934
Gamma 0.50 → Mean Dice: 0.4482
Gamma 0.60 → Mean Dice: 0.6783
Gamma 0.70 → Mean Dice: 0.6879
Gamma 0.80 → Mean Dice: 0.6401
Gamma 0.90 → Mean Dice: 0.5958
Gamma 1.00 → Mean Dice: 0.5471
Gamma 1.10 → Mean Dice: 0.4716
Gamma 1.20 → Mean Dice: 0.4440
Gamma 1.30 → Mean Dice: 0.3843
Gamma 1.40 → Mean Dice: 0.3571
Gamma 1.50 → Mean Dice: 0.3164
Gamma 1.60 → Mean Dice: 0.3009
Gamma 1.70 → Mean Dice: 0.2712
Gamma 1.80 → Mean Dice: 0.2441
Gamma 1.90 → Mean Dice: 0.2359
Gamma 2.00 → Mean Dice: 0.2152
=== Processing N2DL-HeLa ===
Gamma 0.10 → Mean Dice: 0.8865
Gamma 0.20 → Mean Dice: 0.8828
Gamma 0.30 → Mean Dice: 0.8753
Gamma 0.40 → Mean Dice: 0.8623
Gamma 0.50 → Mean Dice: 0.8453
Gamma 0.60 → Mean Dice: 0.8238
Gamma 0.70 → Mean Dice: 0.7966
Gamma 0.80 → Mean Dice: 0.7738
Gamma 0.90 → Mean Dice: 0.7441
Gamma 1.00 → Mean Dice: 0.7164
Gamma 1.10 

(np.float64(0.6000000000000001), np.float64(0.7294237486963051))